In [63]:
from sklearn.feature_extraction import DictVectorizer

from sklearn.metrics import mean_squared_error

import mlflow

import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

import os

import pandas as pd

import pickle

In [30]:
df = pd.read_parquet("../data/df.parquet")
train_df = pd.read_parquet("../data/train_df.parquet")
val_df = pd.read_parquet("../data/val_df.parquet")
test_df = pd.read_parquet("../data/test_df.parquet")

In [31]:
y = "Methyl tert-butyl ether (MTBE)"

In [32]:
X_col = [c for c in df.columns if c not in [y, "sample_date", "station_id"]]

In [33]:
dv = DictVectorizer()

train_dicts = train_df[X_col].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = val_df[X_col].to_dict(orient='records')
X_val = dv.transform(val_dicts)

y_train = train_df[y].values
y_val = val_df[y].values

if not os.path.exists("../models/"):
    os.makedirs("../models/")
with open("../models/preprocessor.b", "wb") as f_out:
    pickle.dump(dv, f_out)

In [34]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [64]:
os.environ["AWS_PROFILE"] = "default"
TRACKING_SERVER_HOST= "ec2-3-90-105-109.compute-1.amazonaws.com"

In [65]:
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")
mlflow.set_experiment("water-quality-prediction")

<Experiment: artifact_location='s3://mlops-zoomcamp-2023/project/2', creation_time=1691216526901, experiment_id='2', last_update_time=1691216526901, lifecycle_stage='active', name='water-quality-prediction', tags={}>

In [39]:
mlflow.xgboost.autolog()

In [40]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

        mlflow.log_artifact("../models/preprocessor.b", artifact_path="preprocessor")     

    return {'loss': rmse, 'status': STATUS_OK}

In [41]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[05:54:55] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:0.51523                           
[1]	validation-rmse:0.56071                           
[2]	validation-rmse:0.78654                           
[3]	validation-rmse:0.84506                           
[4]	validation-rmse:1.10462                           
[5]	validation-rmse:1.36203                           
[6]	validation-rmse:1.40545                           
[7]	validation-rmse:1.63556                           
[8]	validation-rmse:1.84946                           
[9]	validation-rmse:1.87483                           
[10]	validation-rmse:2.06108                          
[11]	validation-rmse:2.08548                          
[12]	validation-rmse:2.24928                          
[13]	validation-rmse:2.27763                          
[14]	validation-rmse:2.29407                          
[15]	validation-rmse:2.31623                          
[

2023/08/07 05:54:57 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.



[05:55:00] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:0.64137                                                    
[1]	validation-rmse:0.76872                                                    
[2]	validation-rmse:0.93029                                                    
[3]	validation-rmse:0.94540                                                    
[4]	validation-rmse:0.98583                                                    
[5]	validation-rmse:0.97021                                                    
[6]	validation-rmse:0.95741                                                    
[7]	validation-rmse:0.94727                                                    
[8]	validation-rmse:0.95506                                                    
[9]	validation-rmse:0.94815                                                    
[10]	validation-rmse:0.96778                                                   
[

2023/08/07 05:55:01 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2023/08/07 05:55:03 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/.local/share/virtualenvs/project-WHDh7pp3/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils."

2023/08/07 05:55:03 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/.local/share/virtualenvs/project-WHDh7pp3/lib/python3.10/site

[05:55:03] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:0.83362                                                     
[1]	validation-rmse:1.22700                                                     
[2]	validation-rmse:1.49925                                                     
[3]	validation-rmse:1.69936                                                     
[4]	validation-rmse:1.84304                                                     
[5]	validation-rmse:1.93551                                                     
[6]	validation-rmse:2.00863                                                     
[7]	validation-rmse:2.05411                                                     
[8]	validation-rmse:2.09170                                                     
[9]	validation-rmse:2.11390                                                     
[10]	validation-rmse:2.12066                                           

...

[05:57:24] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:0.50337                                                      
[1]	validation-rmse:0.52039                                                      
[2]	validation-rmse:0.52665                                                      
[3]	validation-rmse:0.54494                                                      
[4]	validation-rmse:0.55880                                                      
[5]	validation-rmse:0.58369                                                      
[6]	validation-rmse:0.60219                                                      
[7]	validation-rmse:0.62978                                                      
[8]	validation-rmse:0.61734                                                      
[9]	validation-rmse:0.61984                                                      
[10]	validation-rmse:0.64098                                 

2023/08/07 05:57:25 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2023/08/07 05:57:26 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/.local/share/virtualenvs/project-WHDh7pp3/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils."

2023/08/07 05:57:26 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/.local/share/virtualenvs/project-WHDh7pp3/lib/python3.10/site

[05:57:27] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:0.71414                                                      
[1]	validation-rmse:0.79850                                                      
[2]	validation-rmse:0.80682                                                      
[3]	validation-rmse:0.89714                                                      
[4]	validation-rmse:0.88554                                                      
[5]	validation-rmse:0.92345                                                      
[6]	validation-rmse:0.92416                                                      
[7]	validation-rmse:0.98306                                                      
[8]	validation-rmse:0.97609                                                      
[9]	validation-rmse:0.98675                                                      
[10]	validation-rmse:0.99961                                 

2023/08/07 05:57:28 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2023/08/07 05:57:30 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/.local/share/virtualenvs/project-WHDh7pp3/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils."

2023/08/07 05:57:30 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/.local/share/virtualenvs/project-WHDh7pp3/lib/python3.10/site

100%|██████████| 50/50 [02:35<00:00,  3.11s/trial, best loss: 0.6022109159267887]


# Register Model

In [66]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
import mlflow

os.environ["AWS_PROFILE"] = "default"
TRACKING_SERVER_HOST= "ec2-3-90-105-109.compute-1.amazonaws.com"
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")
mlflow.set_experiment("water-quality-prediction")

client = MlflowClient(tracking_uri=f"http://{TRACKING_SERVER_HOST}:5000")

runs = client.search_runs(
    experiment_ids='2',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=1,
    order_by=["metrics.rmse ASC"]
)

runs

[<Run: data=<RunData: metrics={'best_iteration': 5.0,
  'rmse': 0.6022109159267887,
  'stopped_iteration': 54.0,
  'validation-rmse': 0.44144321903481176}, params={'custom_metric': 'None',
  'early_stopping_rounds': '50',
  'learning_rate': '0.13120945483431445',
  'max_depth': '4',
  'maximize': 'None',
  'min_child_weight': '8.762858682537015',
  'num_boost_round': '1000',
  'objective': 'reg:linear',
  'reg_alpha': '0.015743699070586693',
  'reg_lambda': '0.01959215543910407',
  'seed': '42',
  'verbose_eval': 'True'}, tags={'mlflow.log-model.history': '[{"run_id": "d9be3781671e4a90ad194d7f49d6eda6", '
                              '"artifact_path": "model", "utc_time_created": '
                              '"2023-08-07 05:57:02.475778", "flavors": '
                              '{"python_function": {"loader_module": '
                              '"mlflow.xgboost", "python_version": "3.10.12", '
                              '"data": "model.xgb", "env": {"conda": '
            

In [68]:
run_id = runs[0].info.run_id
model_uri = f"runs:/{run_id}/model"
model_name = "water-quality-predictor"
mlflow.register_model(model_uri=model_uri, name=model_name)

Successfully registered model 'water-quality-predictor'.
2023/08/13 07:06:08 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: water-quality-predictor, version 1
Created version '1' of model 'water-quality-predictor'.


<ModelVersion: aliases=[], creation_timestamp=1691910368519, current_stage='None', description='', last_updated_timestamp=1691910368519, name='water-quality-predictor', run_id='d9be3781671e4a90ad194d7f49d6eda6', run_link='', source='s3://mlops-zoomcamp-2023/project/2/d9be3781671e4a90ad194d7f49d6eda6/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [82]:
model_version = 1
stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=stage,
    archive_existing_versions=True
)

<ModelVersion: aliases=[], creation_timestamp=1691910368519, current_stage='Staging', description='', last_updated_timestamp=1691910969961, name='water-quality-predictor', run_id='d9be3781671e4a90ad194d7f49d6eda6', run_link='', source='s3://mlops-zoomcamp-2023/project/2/d9be3781671e4a90ad194d7f49d6eda6/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [83]:
preprocessor_path = '../models'
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path=preprocessor_path)
with open(os.path.join(preprocessor_path, "preprocessor/preprocessor.b"), "rb") as f_in:
    preprocessor = pickle.load(f_in)

In [84]:
model = mlflow.pyfunc.load_model(f"models:/{model_name}/{stage}")

[07:16:14] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
